In [3]:
import pandas as pd
import numpy as ny
#-import findspark
import json
import nltk
import re
#findspark.init()
#from pyspark.sql import SparkSession
#needed

In [ ]:
spark = SparkSession.builder.appName('metadata').getOrCreate()
df = spark.read.csv("metadata.csv")#, low_memory = False)
#reading in dataset in pyspark to see if it better or worse than pandas
#not needed to run

In [ ]:
df.columns #to make sure all data column names od not have any hidden spaces and for future reference
#not needed to run

In [ ]:
df.printSchema() #checking to see how spark prints the data
#not needed to run

In [25]:
dataF = pd.read_csv("metadata.csv", low_memory = False) 
#loading in the file with pandas instead because spark was having a problem withe column names
#needed

In [5]:
dataF.dtypes
#not needed to run

cord_uid             object
sha                  object
source_x             object
title                object
doi                  object
pmcid                object
pubmed_id           float64
license              object
abstract             object
publish_time         object
authors              object
journal              object
mag_id              float64
who_covidence_id     object
arxiv_id             object
pdf_json_files       object
pmc_json_files       object
url                  object
s2_id               float64
dtype: object

In [55]:
cdf2 = dataF[['cord_uid', 'sha', 'source_x', 'title', 'doi', 'pmcid', 'pubmed_id',
       'license', 'abstract', 'publish_time', 'authors', 'journal', 'pdf_json_files', 'pmc_json_files',
       'url']]
cdf2 = cdf2.dropna(axis = 0)
cdf2.reset_index(drop=True,inplace=True)
cdf2.shape
cdf2
#needed to run

,cord_uid,sha,source_x,title,doi,pmcid,pubmed_id,license,abstract,publish_time,authors,journal,pdf_json_files,pmc_json_files,url
0,ug7v899j,d1aafb70c066a2068b02786f8929fd9c900897fb,PMC,Clinical features of culture-proven Mycoplasma...,10.1186/1471-2334-1-6,PMC35282,11472636.0,no-cc,OBJECTIVE: This retrospective chart review des...,2001-07-04,"Madani, Tariq A; Al-Ghamdi, Aisha A",BMC Infect Dis,document_parses/pdf_json/d1aafb70c066a2068b027...,document_parses/pmc_json/PMC35282.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3...
1,02tnwd4m,6b0567729c2143a66d737eb0a2f63f2dce2e5a7d,PMC,Nitric oxide: a pro-inflammatory mediator in l...,10.1186/rr14,PMC59543,11667967.0,no-cc,Inflammatory diseases of the respiratory tract...,2000-08-15,"Vliet, Albert van der; Eiserich, Jason P; Cros...",Respir Res,document_parses/pdf_json/6b0567729c2143a66d737...,document_parses/pmc_json/PMC59543.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5...
2,ejv2xln0,06ced00a5fc04215949aa72528f2eeaae1d58927,PMC,Surfactant protein-D and pulmonary host defense,10.1186/rr19,PMC59549,11667972.0,no-cc,Surfactant protein-D (SP-D) participates in th...,2000-08-25,"Crouch, Erika C",Respir Res,document_parses/pdf_json/06ced00a5fc04215949aa...,document_parses/pmc_json/PMC59549.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5...
3,2b73a28n,348055649b6b8cf2b9a376498df9bf41f7123605,PMC,Role of endothelin-1 in lung disease,10.1186/rr44,PMC59574,11686871.0,no-cc,Endothelin-1 (ET-1) is a 21 amino acid peptide...,2001-02-22,"Fagan, Karen A; McMurtry, Ivan F; Rodman, David M",Respir Res,document_parses/pdf_json/348055649b6b8cf2b9a37...,document_parses/pmc_json/PMC59574.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5...
4,9785vg6d,5f48792a5fa08bed9f56016f4981ae2ca6031b32,PMC,Gene expression in epithelial cells in respons...,10.1186/rr61,PMC59580,11686888.0,no-cc,Respiratory syncytial virus (RSV) and pneumoni...,2001-05-11,"Domachowske, Joseph B; Bonville, Cynthia A; Ro...",Respir Res,document_parses/pdf_json/5f48792a5fa08bed9f560...,document_parses/pmc_json/PMC59580.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37403,q23zf9mj,12d2e87b8795c1839f957b97ebc8c1dfd609b2b0,Medline; PMC,History and Future of Nucleic Acid Amplificati...,10.1159/000496749,PMC6514489,31191192.0,green-oa,The introduction of blood donor screening by v...,2019-01-01,"Roth, Willi Kurt",Transfusion Medicine and Hemotherapy,document_parses/pdf_json/12d2e87b8795c1839f957...,document_parses/pmc_json/PMC6514489.xml.json,https://www.ncbi.nlm.nih.gov/pubmed/31191192/;...
37404,nit5v1wg,142699ba5f2be90b66f9e95fa87e9d8029fddc1e,Medline; PMC,Quantitative structure‐activity relationship a...,10.1002/jmv.24736,PMC7167072,27864902.0,no-cc,A number of human coronaviruses (HCoVs) were r...,2017-02-16,"Elfiky, Abdo A.; Mahdy, Samah M.; Elshemey, Wa...",J Med Virol,document_parses/pdf_json/142699ba5f2be90b66f9e...,document_parses/pmc_json/PMC7167072.xml.json,https://www.ncbi.nlm.nih.gov/pubmed/27864902/;...
37405,3g3apbon,d479fdff7406604cebef7b82284ab7a275e17925,Medline; PMC,The Footprint of Genome Architecture in the La...,10.1371/journal.ppat.1003500,PMC3715407,23874204.0,cc-by,The small size of RNA virus genomes (2-to-32 k...,2013-07-18,"Lauber, Chris; Goeman, Jelle J.; Parquet, Mari...",PLoS Pathog,document_parses/pdf_json/d479fdff7406604cebef7...,document_parses/pmc_json/PMC3715407.xml.json,https://doi.org/10.1371/journal.ppat.1003500; ...
37406,5mv15o9t,ca13097c0bc4ca9abed010fa69c56531749da4dd,Medline; PMC,Zanamivir versus trivalent split virus influen...,10.1111/irv.12301,PMC4353320,25557838.0,cc-by,BACKGROUND: Healthcare workers may be exposed ...,2015-01-04,"Coleman, Brenda L; Fadel, Shaza A; Drews, Stev...",Influenza Other Respir Viruses,document_parses/pdf_json/ca13097c0bc4ca9abed01...,document_parses/pmc_json/PMC4353320.xml.json,https://doi.org/10.1111/irv.12301; https://www...
